In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.1 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 https

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-08-27 21:21:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.48MB/s    in 0.2s    

2022-08-27 21:21:52 (4.48 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

watches_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy", header =True)
watches_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [6]:
print(watches_df.count())

960872


In [7]:
watches_df = watches_df.dropna()
print(watches_df.count())

960679


In [8]:
watches_df = watches_df.dropDuplicates()
print(watches_df.count())

960679


In [9]:
watches_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [10]:
review_id_df = watches_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1006OPQYUMAHS|   43765005|B0000TIIZ0|     529990321| 2008-03-05|
|R1010XUX2AIF10|   44007561|B004KTK2M8|      38406392| 2013-04-11|
|R102DONPK6YEGN|   52805367|B00NHEU9U2|      33550162| 2015-06-23|
|R102N48HXFM9WY|   18449245|B000PSQXWG|     517553139| 2013-02-04|
|R1060WX8UX26JV|   18106128|B0045H3K0G|     989815375| 2013-04-15|
|R106D023EFXYHQ|   44398606|B007900Q9O|     159678416| 2014-09-27|
|R107WJY4V4IZ35|   44609431|B008DZVXVE|     541585709| 2012-12-21|
|R10B275CIJ1QSG|    6507909|B003WPUU0U|     386958467| 2013-10-23|
|R10BIT0LX7BBYD|   22307556|B00DBUVKP0|     578040401| 2015-06-21|
|R10C1EZ2LJLSKN|   18425532|B0088131JK|     326602930| 2015-04-24|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [11]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
review_id_df = review_id_df.withColumn("review_id",col("review_id").cast(StringType())) \
    .withColumn("customer_id",col("customer_id").cast(IntegerType())) \
    .withColumn("product_id",col("product_id").cast(StringType())) \
    .withColumn("product_parent",col("product_parent").cast(IntegerType())) \
    .withColumn("review_date",col("review_date").cast(DateType()))

review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [12]:
products_df = watches_df.select(['product_id', 'product_title'])
products_df.dropDuplicates(['product_id'])
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0000TIIZ0|Timex Women's T47...|
|B004KTK2M8|Leather Watchband...|
|B00NHEU9U2|Fanmis Men's Wate...|
|B000PSQXWG|Nautica Men's N14...|
|B0045H3K0G|Columbia Men's CT...|
|B007900Q9O|Citizen Quartz Go...|
|B008DZVXVE|Swiss Legend Men'...|
|B003WPUU0U|G-SHOCK The GA 10...|
|B00DBUVKP0|Citizen Men's JY8...|
|B0088131JK|ADIDAS Performanc...|
+----------+--------------------+
only showing top 10 rows



In [13]:
products_df = products_df.withColumn("product_id",col("product_id").cast(StringType())) \
    .withColumn("product_title",col("product_title").cast(StringType()))

products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [14]:
customers_df = watches_df.select(['customer_id'])
customers_df.show(10)

+-----------+
|customer_id|
+-----------+
|   43765005|
|   44007561|
|   52805367|
|   18449245|
|   18106128|
|   44398606|
|   44609431|
|    6507909|
|   22307556|
|   18425532|
+-----------+
only showing top 10 rows



In [23]:
customers_df = customers_df.groupby('customer_id').count()
customers_df.show(10)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    1567510|    1|
|   19502021|    1|
|   12819130|    1|
|   35329257|    1|
|     108460|    1|
|    5453476|    1|
|   29913055|    1|
|   30717305|    1|
|    1570030|    1|
|   19032020|    1|
+-----------+-----+
only showing top 10 rows



In [24]:
customers_df = customers_df.withColumnRenamed('count', 'customer_count')
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    1567510|             1|
|   19502021|             1|
|   12819130|             1|
|   35329257|             1|
|     108460|             1|
|    5453476|             1|
|   29913055|             1|
|   30717305|             1|
|    1570030|             1|
|   19032020|             1|
+-----------+--------------+
only showing top 10 rows



In [25]:
customers_df = customers_df.withColumn("customer_id",col("customer_id").cast(IntegerType())) \
    .withColumn("customer_count",col("customer_count").cast(IntegerType()))

customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [26]:
vine_table_df = watches_df.select(["review_id", "star_rating","helpful_votes", "total_votes", "vine"])
vine_table_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1006OPQYUMAHS|          1|            4|          5|   N|
|R1010XUX2AIF10|          5|            0|          0|   N|
|R102DONPK6YEGN|          5|            0|          0|   N|
|R102N48HXFM9WY|          5|            0|          0|   N|
|R1060WX8UX26JV|          2|            0|          0|   N|
|R106D023EFXYHQ|          5|            0|          0|   N|
|R107WJY4V4IZ35|          5|            0|          0|   N|
|R10B275CIJ1QSG|          4|            0|          0|   N|
|R10BIT0LX7BBYD|          4|            1|          2|   N|
|R10C1EZ2LJLSKN|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [27]:
vine_table_df = vine_table_df.withColumn("review_id",col("review_id").cast(StringType())) \
    .withColumn("star_rating",col("star_rating").cast(IntegerType())) \
    .withColumn("helpful_votes",col("helpful_votes").cast(IntegerType())) \
    .withColumn("total_votes",col("total_votes").cast(IntegerType())) \
    .withColumn("vine",col("vine").cast(StringType()))

vine_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [28]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://:5432/my_data_class_db2"
config = {"user":"root",
          "password": "",
          "driver":"org.postgresql.Driver"}

In [29]:
# Write DataFrame to table

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [30]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [31]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [32]:
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)